In [1]:
import pandas as pd

In [2]:
cpuc = pd.read_csv('data/cpuc_with_budget_and_resource.csv')

C:\Users\audre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,12,17,19,22,23,24,27,47,48,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def aggregate_me(df, by, lg, column_list):
    
    # making df usable

    df[by] = df[by].str.upper()
    
    #filtering for local government

    if lg: 

        lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

        df = df[df['Grouping'].isin(lglist)]


    # creating a total programs column

    prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
    #need to count the number of times a city/county shows up 

    e,f = [list(c) for c in zip(*prg)]

    ccp = pd.Series(e) # made a series so can use value_counts

    ccp = ccp.str.upper()

    tp = ccp.value_counts(dropna = False) #total programs per city
    
    # creating a total claims column
    
    df['Total Claims'] = [1]*len(df)
    
    #aggregate!
    relevant_columns = column_list + ['Total Claims']
    agg = df.groupby(by).sum()[relevant_columns]
    agg['Total Programs'] = tp
    
    # Resource/OBF budget as percentage of total budget
    obf_budget = df.groupby(['SiteCity', 'OBF_Flag']).Budget.sum().unstack().fillna(0).loc[:, True]
    resource_budget = df.groupby(['SiteCity', 'Resource_Flag']).Budget.sum().unstack().fillna(0).loc[:, 1]
    agg['% OBF Budget'] = (obf_budget/agg['Budget'])*100
    agg['% Resource Budget'] = (resource_budget/agg['Budget'])*100
    
    return(agg.reset_index())

In [4]:
def no_dupes(df):
    
    #creating df of number of times a claim repeats

    repeats = df['ClaimID'].value_counts().to_frame().reset_index()

    #getting list of duplicate claims
    
    all_dupes = repeats[repeats['ClaimID']>1]
    dupes = all_dupes['index'].to_list()
   

    #getting one index value per duplicate claim

    goodi=[]
    for e in dupes:
        goodi.append(min(df[df['ClaimID'].isin([e])].index.to_list()))
    
    # full list of duplicate claim indicies

    alli=[]
    for e in dupes:
        alli = alli + df[df['ClaimID'].isin([e])].index.to_list()


    # take goodi from alli
    badi = [i for i in alli if i not in goodi]

    #drop duplicate indicies
    return(df.drop(index=badi).reset_index())

In [5]:
copy = cpuc.copy()

copy.drop(columns='Unnamed: 0', inplace=True)

copy = copy.drop_duplicates().reset_index()

unique_claims = no_dupes(copy)

In [6]:
city_ag = aggregate_me(unique_claims, 'SiteCity', True, ['TotalFirstYearGrosskWh', 'TotalFirstYearGrossTherm','TotalLifecycleGrosskWh', 'TotalLifecycleGrossTherm', 'Budget'])

<ipython-input-3-804daff755ff>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total Claims'] = [1]*len(df)


In [7]:
city_ag

,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Total Claims,Total Programs,% OBF Budget,% Resource Budget
0,ACAMPO,48664.726740,-148.890330,5.839767e+05,-1786.683960,17684.568509,6,1,0.000000,100.0
1,ADELANTO,13576.844232,-30.037266,6.788422e+04,-150.186330,27320.063231,4,1,0.000000,100.0
2,ALAMO,73049.392600,118.645240,6.850000e+05,-1279.097025,48785.542683,15,1,88.044896,100.0
3,ALBANY,153765.922000,-785.076334,8.422006e+05,-4288.070280,60926.193528,35,2,0.000000,100.0
4,ALHAMBRA,21630.949392,-32.172462,1.202334e+05,-178.879938,48465.447601,7,1,0.000000,100.0
...,...,...,...,...,...,...,...,...,...,...
532,WOODLAND,526910.057920,-288.682189,5.986637e+06,-3137.043203,278715.826268,358,2,9.537819,100.0
533,YORBA LINDA,100589.028000,-23.217678,1.207068e+06,-278.612136,230594.633505,12,1,100.000000,100.0
534,YOSEMITE NATIONAL PARK,8894.000000,-51.060000,7.826720e+04,-449.328000,18234.382541,4,1,0.000000,100.0
535,YOUNTVILLE,16598.672480,-37.547000,1.580779e+05,-258.096560,4901.835174,57,2,0.000000,100.0


In [ ]:
city_ag

In [10]:
city_ag.to_csv('data/city_ag_2.csv')

------
-----


### The journey to classify all programs into a county....cannot do this manually because there are more than 50,000 claims that are local government and countyless....

In [24]:
unique_claims[unique_claims['County'].isnull()]['Grouping'].value_counts()

Local Government (Counties)                40859
Local Government ()                        17142
Education ()                                3053
State ()                                     471
Education (Higher Education CCC)             366
Education (Higher Education UC/CSU)          202
Education (K-12)                             159
Local Government (Cities)                     65
State (Correctional Facilities)               56
Local Government                              13
Education (Higher Education CSU)              13
State (Correctional Facilities)               13
Education (Higher Education UC/CSU/CCC)        3
Federal ()                                     1
Name: Grouping, dtype: int64

### will have to find a city to county mapping dataset and then merge 

In [25]:
uscities = pd.read_csv('data/uscities.csv', usecols = ['city', 'state_id', 'county_name'])

In [34]:
cacities = uscities[uscities['state_id']=='CA']

In [35]:
cacities.city = cacities.city.str.upper()

C:\Users\audre\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [37]:
cacities.head()

,city,state_id,county_name
1,LOS ANGELES,CA,Los Angeles
12,SAN FRANCISCO,CA,San Francisco
14,SAN DIEGO,CA,San Diego
20,RIVERSIDE,CA,Riverside
26,SACRAMENTO,CA,Sacramento


In [39]:
unique_claims_county = unique_claims.merge(cacities.drop(columns=['state_id']), left_on='SiteCity', right_on='city',how='left')

In [42]:
unique_claims_county[unique_claims_county['SiteCity'] != unique_claims_county['city']].SiteCity.value_counts()

PASO ROBLES      2661
VENTURA          1775
MC FARLAND       1266
SAN JUAN CAPO    1018
VALENCIA          998
                 ... 
GARDEN VALLEY       1
SANTA MARGAR        1
GREENWOOD           1
ORLEANS             1
SKY FOREST          1
Name: SiteCity, Length: 231, dtype: int64

In [51]:
cacities.city.sort_values(ascending=True).reset_index().iloc[992:996]

,index,city
992,318,PASADENA
993,13569,PASATIEMPO
994,26168,PASKENTA
995,2098,PATTERSON


can't use above mapping because doesn't include many large cities that should easily be accounted for